In [1]:
import numpy as np
import copy
import json
import math
from glob import glob
import scipy.spatial.distance as sciDist
from tqdm import tqdm
import requests
import time
import itertools
import random
import os
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import islice
from PIL import Image
import re
from tqdm import tqdm


# Headless simulator version
index = 7 # local server index 
#API_ENDPOINT = 'http://localhost:400' + str(index) + '/simulation' # NOT THE LS VERSION
API_ENDPOINT = 'http://localhost:400' + str(1) + '/simulation' # NOT THE LS VERSION
HEADERS = {"Content-Type": "application/json"}
batchCount = 25 # Send this number of samples to MotionGen each time 
speedscale = 1
steps = 360
minsteps = int(steps*20/360)

# Search for "stephenson" or "watt" in the JSON file and collect their names and indices
json_path = "KV_468_062324.json"
with open(json_path, "r") as f:
    data = json.load(f)

matches = []
for idx, name in enumerate(data):
    lname = name.lower()
    if "step" in lname or "watt" in lname:
        matches.append([name, idx])
print(matches)


# Things for 4 bar 
mechType = index
types = ['RRRR', 'RRRP', 'RRPR', 'PRPR']
for mechanism in matches:
    types.append(mechanism[0])
print(types)
# ['RRRR', 'RRRP', 'RRPR', 'PRPR', 'Watt2T1A1', 'Steph1T3', 'Watt1T1A1', 'Steph3T2A2', 'Watt1T3A2', 'Steph2T2A1', 'Steph3T1A2', 'Watt2T2A2', 'Watt1T2A1', 'Steph2T1A1', 'Steph3T2A1', 'Watt1T1A2', 'Watt2T1A2', 'Steph1T2', 'Steph1T1', 'Watt1T3A1', 'Watt1T2A2', 'Watt2T2A1', 'Steph3T1A1', 'Steph2T1A2', 'Steph2T2A2']

# Create a directory in outputs-6bar for each type in types
'''
for t in types:
    if t in ('RRRR', 'RRRP', 'RRPR', 'PRPR'):
        continue
    dir_path = os.path.abspath("./outputs-6bar/" + t)
    os.makedirs(dir_path, exist_ok=True)'''



typeIndex = [49, 64, 155, 175] # to avoid confusion from any other type
for mechanism in matches:
    typeIndex.append(mechanism[1])

print(typeIndex)
# [49, 64, 155, 175, 0, 2, 19, 26, 37, 39, 73, 82, 90, 110, 111, 125, 133, 135, 137, 139, 149, 163, 169, 176, 177]


#couplerCurveIndex = [4, 4, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]


#confirm that all c values are 1 at index 7 for 6 bars

bsi = "BSIdict_468_062324_3.json"
with open(bsi, "r") as b:
    data = json.load(b)

for key in types:
    if key in data:
        c_value = (data)[key].get("c")
        print(key, c_value)

# Update couplerCurveIndex so that for each type, its value is the index of the number 1 in the c_value list from BSIdict
couplerCurveIndex = []
for key in types:
    if key in data:
        c_value = data[key].get("c")
        if isinstance(c_value, list) and 1 in c_value:
            couplerCurveIndex.append(c_value.index(1))
        else:
            couplerCurveIndex.append(4)  # fallback default
    else:
        couplerCurveIndex.append(4)      # fallback default
print("Updated couplerCurveIndex:", couplerCurveIndex)
# Updated couplerCurveIndex: [4, 4, 4, 4, 7, 7, 7, 7, 6, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 6, 7, 7, 4, 7, 4]


savePointNumber = [5, 6, 7, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8] #len of c arrat acording to BSIdict
needAddtional = [False, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False] # notice that when true, some of the points can change its position randomly 
#initStates = np.load("./npy-inputs/" + 'RandPos-.npy')

initStates = np.load("/gpfs/scratch/raytang/walking-dataset-generator-old-simulator/Image-method-Synthesis-main/npy-inputs/RandPos-.npy")

errCtr = 0
batch = []
batchSaveStr = []
batchSaveNpyStr = []


# The transformation 
#np.save(saveDir + name + ' ' + types[index], param)
saveDir = os.path.abspath("./outputs-6bar/" + types[index] )
saveDirNpy = os.path.abspath("./outputs-6bar/" + types[index] + "-npy")
print(saveDir, saveDirNpy)



[['Watt2T1A1', 0], ['Steph1T3', 2], ['Watt1T1A1', 19], ['Steph3T2A2', 26], ['Watt1T3A2', 37], ['Steph2T2A1', 39], ['Steph3T1A2', 73], ['Watt2T2A2', 82], ['Watt1T2A1', 90], ['Steph2T1A1', 110], ['Steph3T2A1', 111], ['Watt1T1A2', 125], ['Watt2T1A2', 133], ['Steph1T2', 135], ['Steph1T1', 137], ['Watt1T3A1', 139], ['Watt1T2A2', 149], ['Watt2T2A1', 163], ['Steph3T1A1', 169], ['Steph2T1A2', 176], ['Steph2T2A2', 177]]
['RRRR', 'RRRP', 'RRPR', 'PRPR', 'Watt2T1A1', 'Steph1T3', 'Watt1T1A1', 'Steph3T2A2', 'Watt1T3A2', 'Steph2T2A1', 'Steph3T1A2', 'Watt2T2A2', 'Watt1T2A1', 'Steph2T1A1', 'Steph3T2A1', 'Watt1T1A2', 'Watt2T1A2', 'Steph1T2', 'Steph1T1', 'Watt1T3A1', 'Watt1T2A2', 'Watt2T2A1', 'Steph3T1A1', 'Steph2T1A2', 'Steph2T2A2']
[49, 64, 155, 175, 0, 2, 19, 26, 37, 39, 73, 82, 90, 110, 111, 125, 133, 135, 137, 139, 149, 163, 169, 176, 177]
RRRR [0, 0, 0, 0, 1]
RRRP [0, 0, 0, 0, 1, 0]
RRPR [0, 0, 0, 0, 1, 0, 0]
PRPR [0, 0, 0, 0, 1]
Watt2T1A1 [0, 0, 0, 0, 0, 0, 0, 1]
Steph1T3 [0, 0, 0, 0, 0, 0, 0, 1]

In [2]:
# good old ones 

def isValid(seq):
    if len(seq.shape) == 2:
        isVal = np.var(seq[:,0]) <= 5e-3 and np.var(seq[:,1]) <= 5e-3
    else:
        isVal = len(seq) == 0 or np.var(seq) <= 5e-3

    if isVal:
        return False
    else:
        return True


def get_pca_inclination(qx, qy, ax=None, label=''):
    """ Performs the PCA
        Return transformation matrix
    """
    cx = np.mean(qx)
    cy = np.mean(qy)
    covar_xx = np.sum((qx - cx)*(qx - cx))/len(qx)
    covar_xy = np.sum((qx - cx)*(qy - cy))/len(qx)
    covar_yx = np.sum((qy - cy)*(qx - cx))/len(qx)
    covar_yy = np.sum((qy - cy)*(qy - cy))/len(qx)
    covar = np.array([[covar_xx, covar_xy],[covar_yx, covar_yy]])
    eig_val, eig_vec= np.linalg.eig(covar)

    # Inclination of major principal axis w.r.t. x axis
    if eig_val[0] > eig_val[1]:
        phi= np.arctan2(eig_vec[1,0], eig_vec[0,0])
    else:
        phi= np.arctan2(eig_vec[1,1], eig_vec[0,1])

    return phi


def get_normalize_curve(jd, steps=None, rotations=1, normalize=True, transformParas=None):
    jd = np.array(jd)
    joint_data_n, x_mean, y_mean, denom, phi = [], None, None, None, None
    if isValid(jd):
        if steps:
            sample_indices = np.linspace(0, jd.shape[0]-1, steps, dtype=np.int32)
            jd = jd[sample_indices,:]
        if normalize:
            if not transformParas:
                x_mean = np.mean(jd[:,0], axis=0, keepdims=True)
                y_mean = np.mean(jd[:,1], axis=0, keepdims=True)
            else:
                x_mean, y_mean, denom, phi = transformParas
            jd[:,0] = jd[:,0] - x_mean
            jd[:,1] = jd[:,1] - y_mean

            if not transformParas:
                denom = np.sqrt(np.var(jd[:,0], axis=0, keepdims=True) + np.var(jd[:,1], axis=0, keepdims=True))
                denom = np.expand_dims(denom, axis=1)
            jd = jd / denom
            t = 0
        if not transformParas:
            phi = -get_pca_inclination(jd[:,0], jd[:,1])
        jd[:,0], jd[:, 1] = rotate_curve(jd, phi)
        for tt in range(rotations):
            joint_data_n.append(jd.copy())
            if rotations > 1:
                jd[:,0], jd[:,1] = rotate_curve(jd, t)
                t = 2*np.pi/rotations

    return joint_data_n, x_mean, y_mean, denom, phi


def rotate_curve(cur, theta):
    cpx = cur[:,0]*np.cos(theta) - cur[:,1]*np.sin(theta)
    cpy = cur[:,0]*np.sin(theta) + cur[:,1]*np.cos(theta)
    return cpx, cpy


def digitize_seq(nums, minlim, maxlim, bin_size=64):
    bins = np.linspace(minlim, maxlim, bin_size-1)
    nums_indices = np.digitize(nums, bins)
    return nums_indices


def get_normalize_joint_data_wrt_one_curve(joint_data, ref_ind = 4):
    ''' input s = [num_curves, num_points, 2]
    '''
    joint_data_n = []
    s = np.array(joint_data)
    if isValid(s[ref_ind]):
        x_mean = np.mean(s[ref_ind:ref_ind+1,:,0], axis=1, keepdims=True)
        y_mean = np.mean(s[ref_ind:ref_ind+1,:,1], axis=1, keepdims=True)
        s[:,:,0] = s[:,:,0] - x_mean
        s[:,:,1] = s[:,:,1] - y_mean
        denom = np.sqrt(np.var(s[ref_ind:ref_ind+1,:,0], axis=1, keepdims=True) + np.var(s[ref_ind:ref_ind+1,:,1], axis=1, keepdims=True))
        denom = np.expand_dims(denom, axis=2) #is this scale? 
        s = s / denom
        phi = -get_pca_inclination(s[ref_ind:ref_ind+1,:,0], s[ref_ind:ref_ind+1,:,1])
        for i in range(s.shape[0]):
            s[i,:,0], s[i,:,1] = rotate_curve(s[i], phi)
    else:
        return s, [None, None, None, None], False

    # s has a shape of (j_num, state, dim)
    return s, [x_mean[0][0], y_mean[0][0], denom[0][0][0], phi], True # tx, ty, scaling, rotation angle 


##############################################################################################
# There are some other necessary transformations. (x_mean, y_mean, phi, denom) are from get_normalize_curve. 
##############################################################################################
def get_image_from_point_cloud(points, xylim, im_size, inverted = True, label=None):
    mat = np.zeros((im_size, im_size, 1), dtype=np.uint8)
    x = digitize_seq(points[:,0], -xylim, xylim, im_size)
    if inverted:
        y = digitize_seq(points[:,1]*-1, -xylim, xylim, im_size)
        mat[y, x, 0] = 1
    else:
        y = digitize_seq(points[:,1], -xylim, xylim, im_size)
        mat[x, y, 0] = 1
    return mat


def process_mech_102723(jointData, ref_ind, im_size = 64, xylim = 3.5, inverted = True, swapAxes = True):
    paras = None

    # It is possible the jointData format is (angles, joint, (x, y)). 
    # You should put a True if this happens. (This is how files are saved).
    # I literally don't understand why I saved jointData with a shape of (angles, joint, (x, y)) 
    if swapAxes:
        jointData = np.swapaxes(jointData, 0, 1)

    # This converts all 
    jointData, paras, success = get_normalize_joint_data_wrt_one_curve(jointData, ref_ind= ref_ind)

    # jointData format from now on becomes np.array with a shape of (joint, curve_length, dimension)
    jointData = np.array(jointData)

    if success:
        # get binaryImage 
        jd = jointData[ref_ind]
        mat = get_image_from_point_cloud(jd, xylim=xylim, im_size=im_size, inverted=inverted)
        return mat, paras, success
    else: 
        return None, None, success


def calc_dist(coord):
    # Calculate differences using broadcasting
    diffs = coord[:, np.newaxis, :] - coord[np.newaxis, :, :]
    squared_dists = np.sum(diffs ** 2, axis=2)

    # Extract the upper triangle indices where i < j
    i, j = np.triu_indices(len(coord), k=1)
    dist_arr = np.sqrt(squared_dists[i, j])
    dist_arr = dist_arr/min(dist_arr)
    return np.round(dist_arr, 2)

In [3]:
############################################################################################## 
# An even better version of normalization method (it can be slower sometimes)
# Uses reflection correction (contributed by the paper Geometric Invariant Curve and Surface Normalization)  
# Thanks to Zhijie who noticed these two can be used together. 
##############################################################################################

def rotate_curve(curve, phi):
    infunction_scale = 100
    # curve is scaled 100 times for numerical accuracy 
    # incoming curve shape: (n,2)
    x = curve[:, 0] * infunction_scale
    y = curve[:, 1] * infunction_scale
    # Compute the rotated coordinates
    x_rotated = x * np.cos(phi) - y * np.sin(phi)
    y_rotated = x * np.sin(phi) + y * np.cos(phi)
    # Combine the rotated coordinates into a new curve
    rotated_curve = np.column_stack((x_rotated, y_rotated))
    return rotated_curve/infunction_scale


def center_data(X):
    """ Center the data by subtracting the mean of each column.
        Return the translated X and the translation matrix 
    """
    m = np.mean(X, axis=0) # (n, 2)
    return X - m, np.matrix([[1, 0, -m[0]], [0, 1, -m[1]], [0, 0, 1]]) # equal to XP this is a translation matrix tranposed  


def scale_data(X, scaling = 0): 
    """ Scale the data according to two different metrics 
        If scaling == 0 (default), scaling method is normalization (average distance 1)
        If otherwise, scaling method is standardization to a certain scale 
        Return the scaled X, and the scaling matrix. 
    """
    if scaling == 0:
        # use variance. 
        denom = np.sqrt(np.var(X[:,0]) + np.var(X[:,1]))
        scaled_curve = X /denom
        ScaleMat = np.matrix([[1/denom, 0, 0], [0, 1/denom, 0], [0, 0, 1]])
    else:
        # Compute the maximum distance from the origin 
        max_distance = np.max(np.linalg.norm(X, axis=1))
        scaled_curve = X * scaling / max_distance
        ScaleMat = np.matrix([[scaling/max_distance, 0, 0], [0, scaling/max_distance, 0], [0, 0, 1]])
    return scaled_curve, ScaleMat


def rotate_data(X, tol = 1e-4, randinit = False): 
    """ Performs the PCA and determines rotation angle phi 
        More precisely it is snapping the greatest principal axis to the X-axis. 
        Return the rotated X and the rotation matrix 
    """
    phiInit = 0
    if randinit:
        phiInit = np.random.rand() * math.pi * 2 

    rotationMatInit = np.matrix([
        [np.cos(phiInit), -np.sin(phiInit), 0], 
        [np.sin(phiInit), np.cos(phiInit), 0],
        [0, 0, 1] 
    ])

    X0 = rotate_curve(X, phiInit)
    cx = np.mean(X0[:,0])
    cy = np.mean(X0[:,1])
    covar_xx = np.sum((X0[:,0] - cx)*(X0[:,0] - cx))/X0.shape[0]
    covar_xy = np.sum((X0[:,0] - cx)*(X0[:,1] - cy))/X0.shape[0]
    covar_yx = np.sum((X0[:,1] - cy)*(X0[:,0] - cx))/X0.shape[0]
    covar_yy = np.sum((X0[:,1] - cy)*(X0[:,1] - cy))/X0.shape[0]
    covar = np.array([[covar_xx, covar_xy],[covar_yx, covar_yy]])

    if np.abs(np.linalg.det(covar)) < tol:
        phi = 0 # why rotate anyway? 
    else:
        eig_val, eig_vec= np.linalg.eig(covar) 
        # Inclination of major principal axis w.r.t. x axis
        # Enforcing the cross-product of the two eigenvectors to be greater than 0. 
        # Not necessary, but it looks clean to do so. 
        # Eigenvector matrix: [a, b], det = crossproduct of b x a
        if np.linalg.det(eig_vec) > 0:
            eig_vec[0,:] = -eig_vec[0,:] # enforcing a x b > 0 
        if eig_val[0] > eig_val[1]:
            phi= np.arctan2(eig_vec[1,0], eig_vec[0,0])
        else:
            phi= np.arctan2(eig_vec[1,1], eig_vec[0,1])
    rotated_curve = rotate_curve(X0, phi)
    rotationMat = np.matrix([
        [np.cos(phi), -np.sin(phi), 0], 
        [np.sin(phi), np.cos(phi), 0],
        [0, 0, 1] 
    ])

    return rotated_curve, np.matmul(rotationMat, rotationMatInit)


def reflect_data(X):
    """ Computes the third order moment and determines the reflections 
        The data must be rotated before this step. 

    """
    # Reflection normalization 
    x_scaled = X[:, 0]
    y_scaled = X[:, 1]

    # see paper Geometric Invariant Curve and Surface Normalization
    # compute the 3rd-order moments 
    m12 = np.sum((x_scaled**1)*(y_scaled**2))
    m21 = np.sum((x_scaled**2)*(y_scaled**1))
    signm12 = np.sign(m12)
    signm21 = np.sign(m21)
    if np.abs(signm12) < 1e-5:
        signm12 = 1
    if np.abs(signm21) < 1e-5:
        signm21 = 1

    reflectionMat = np.array(
        [[signm12, 0],
         [0, signm21]]
    ) 

    if np.abs(m12) > np.abs(m21):
        reflectionMat = np.matmul(np.array([[0,1],[1,0]]), reflectionMat)

    reflected_Curve = np.matmul(reflectionMat, np.array(X).T).T
    reflectionMat = np.matrix(
        [[reflectionMat[0,0], reflectionMat[0,1], 0], 
         [reflectionMat[1,0], reflectionMat[1,1], 0], 
         [0, 0, 1]
        ]
    ) 

    return reflected_Curve, reflectionMat


# Digitization 
def digitize_seq(nums, minlim, maxlim, bin_size=64):
    bins = np.linspace(minlim, maxlim, bin_size-1)
    nums_indices = np.digitize(nums, bins)
    return nums_indices


def get_image_from_point_cloud(points, xylim, im_size, inverted = True, label=None):
    mat = np.zeros((im_size, im_size, 1), dtype=np.uint8)
    x = digitize_seq(points[:,0], -xylim, xylim, im_size)
    if inverted:
        y = digitize_seq(points[:,1]*-1, -xylim, xylim, im_size)
        mat[y, x, 0] = 1
    else:
        y = digitize_seq(points[:,1], -xylim, xylim, im_size)
        mat[x, y, 0] = 1
    return mat


# Returns normalized curve, Transformation matrix, and Success (if determinant is greater than tolerance for numerical stability)
def normalize_data_122223(X, scaling = 0, tol = 1e-8, maxiter = 2):
    X1, M1 = center_data(X) 
    X1, M2 = scale_data(X1, scaling = scaling)
    X1, M3 = rotate_data(X1)
    X1, M4 = reflect_data(X1)
    M = M4*M3*M2*M1 # This is the transformation matrix 

    detVal = np.abs(np.linalg.det(M))
    if detVal*scaling < tol:
        for i in range(maxiter):
            X1, M1 = center_data(X1)
            X1, M2 = scale_data(X1, scaling = scaling)
            X1, M3 = rotate_data(X1, randinit= True)
            X1, M4 = reflect_data(X1)
            if np.abs(np.linalg.det(M)) > tol or detVal*10 < np.abs(np.linalg.det(M)):
                break
    return X1, M4*M3*M2*M1, np.abs(np.linalg.det(M)) > tol


def matmul_jd(jd, mat):
    # input should be (..., ,2)
    # the operation: 
    jd = np.array(jd)
    oldshape = jd.shape
    njd= np.reshape(jd, (-1, 2))
    hc1= np.ones((njd.shape[0], 1))
    njd=np.matrix(np.concatenate([njd, hc1], axis = 1)).transpose()
    njd=np.array((mat*njd).transpose())[:,0:2].reshape(oldshape)
    return njd


# Processing 
# THIS IS THE MAIN FUNCTION YOU NEED (second version) 
# Input is the mechanism and coupler point index (ref_ind), Output is matImg, [<two transformation matrix instead of six parameters>], and Success. 
# NOTICE 1: the parameter has SIX parameters, not four (x, y translation, angle and scale). The addtional parameters are reflectionMat, seqReversed (refrlection matrix and whether or not the sequence is reverse in order)
# NOTICE 2: the transformation matrices are numpy arrays. You need to convert them to lists before converting them into .json format objects. 
# I suggest you save the a, b, c, d, e, f values in the matrix. 
# A transformation matrix is shaped as: 
# [[a, c, e]
#  [b, d, f]
#  [0, 0, 1]]
def process_mech_051524(jointData, ref_ind, im_size = 64, xylim = 3.5, inverted = True):
    # New JD shape is always (states, joints, dimensions)
    paras = None
    # get matrices according to curve. 
    nc, mat, success = normalize_data_122223(jointData[:,ref_ind,:], scaling = 3.5)
    NR_MG = mat # transform original position to normalized position # NR_MG * jointData = normalized joint data
    MG_NR = np.linalg.inv(NR_MG) 

    if success:
        # get binaryImage # rohit's choice on what to save 
        paras = NR_MG
        matImg = get_image_from_point_cloud(nc, xylim=xylim, im_size=im_size, inverted=inverted)
        return matImg, paras, success
    else: 
        return None, None, success

    
def calc_dist(coord):
    # Calculate differences using broadcasting
    diffs = coord[:, np.newaxis, :] - coord[np.newaxis, :, :]
    squared_dists = np.sum(diffs ** 2, axis=2)

    # Extract the upper triangle indices where i < j
    i, j = np.triu_indices(len(coord), k=1)
    dist_arr = np.sqrt(squared_dists[i, j])
    dist_arr = dist_arr/min(dist_arr)
    return np.round(dist_arr, 2)

In [4]:
def format_floats_to_string(float_list):
    formatted_strings = [f"{value:.3f}" for value in float_list]
    result_string = ' '.join(formatted_strings)
    return ' ' + result_string

# Example usage
float_values = [3920, -1.000e-03, 5.393e+00, 1.748e+00, -8.786e+00, 2.079e+00, 5.023e+00, -4.854e+00, 4.230e-01, 5.406e+00, -1.882e+00, -1.388e+00, 0.241, -0.285, 0.455, 0.899, 0.0, 1.0]

formatted_string = format_floats_to_string(float_values)
print(formatted_string)



 3920.000 -0.001 5.393 1.748 -8.786 2.079 5.023 -4.854 0.423 5.406 -1.882 -1.388 0.241 -0.285 0.455 0.899 0.000 1.000


In [ ]:
import multiprocessing as mp
from functools import partial
import os
import numpy as np
import requests
import json
import time
import re
from tqdm import tqdm
from PIL import Image
from scipy.ndimage import gaussian_filter1d
from shapely.geometry import LineString
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt

# Your existing functions remain unchanged
def has_sharp_edges(coords, curvature_threshold=15.0, smoothing_sigma=1.0):
    if smoothing_sigma > 0:
        x = gaussian_filter1d(coords[:,0], sigma=smoothing_sigma)
        y = gaussian_filter1d(coords[:,1], sigma=smoothing_sigma)
    else:
        x = coords[:,0]
        y = coords[:,1]

    dx = np.gradient(x)
    dy = np.gradient(y)
    ddx = np.gradient(dx)
    ddy = np.gradient(dy)

    curvature = np.abs(dx * ddy - dy * ddx) / (dx**2 + dy**2)**1.5
    return np.any(curvature > curvature_threshold), curvature

def resample_uniform(points, M):
    dif = np.diff(points, axis=0)
    seglen = np.sqrt((dif**2).sum(axis=1))
    s = np.concatenate(([0], np.cumsum(seglen)))
    total = s[-1]
    if total == 0:
        return np.repeat(points[:1], M, axis=0)
    s_uniform = np.linspace(0, total, M)
    x = np.interp(s_uniform, s, points[:,0])
    y = np.interp(s_uniform, s, points[:,1])
    return np.column_stack((x, y))

def pca_align(points):
    pts = points - np.mean(points, axis=0)
    U, S, VT = np.linalg.svd(pts, full_matrices=False)
    pcs = VT.T
    rotated = pts @ pcs
    return rotated, pcs

def detect_flat_via_pca(points, K_keep=12, N_resample=512, eps_slope=0.0075, min_fraction=1/4):
    pts = resample_uniform(points, N_resample)
    rotated, pcs = pca_align(pts)
    dy = np.gradient(rotated[:,1])
    flat_mask = np.abs(dy) < eps_slope
    
    from itertools import groupby
    runs = [sum(1 for _ in g) for val,g in groupby(flat_mask) if val]
    max_run = max(runs) if runs else 0
    flat_fraction = max_run / len(flat_mask)
    is_flat_enough = flat_fraction >= min_fraction
    
    first_pc = pcs[:,0]
    angle = np.arctan2(first_pc[1], first_pc[0])

    max_run = 0
    run_indices = []
    current_run = []

    for i, is_flat in enumerate(flat_mask):
        if is_flat:
            current_run.append(i)
            if len(current_run) > max_run:
                max_run = len(current_run)
                run_indices = current_run.copy()
        else:
            current_run = []

    return {
        'flat_fraction': float(flat_fraction),
        'is_flat_enough': bool(is_flat_enough),
        'orientation_rad': float(angle),
        'orientation_deg': float(np.degrees(angle)),
        'max_run': int(max_run),
        'N': int(len(flat_mask)),
        'flat_indices': np.array(run_indices, dtype=int), 
        'pts': pts,
    }

def ground_clearance_check(path_coords, joint_coords, flat_indices):
    if len(flat_indices) == 0:
        return False
    flat_pts = path_coords[flat_indices]
    m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)

    d_path = path_coords[:,1] - (m*path_coords[:,0] + b)
    majority_sign = np.sign(np.sum(d_path))
    if majority_sign == 0:
        majority_sign = 1

    d_joints = joint_coords[:,1] - (m*joint_coords[:,0] + b)
    all_clear = np.all(np.sign(d_joints) == majority_sign)
    return all_clear

def save_flat_fit_with_joints(path_coords, flat_indices, joint_coords, save_dir, filename):
    if len(flat_indices) == 0:
        return
    
    flat_pts = path_coords[flat_indices]
    m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)
    x_line = np.linspace(path_coords[:,0].min(), path_coords[:,0].max(), 200)
    y_line = m * x_line + b

    d_path = path_coords[:,1] - (m*path_coords[:,0] + b)
    majority_sign = np.sign(np.sum(d_path))
    if majority_sign == 0:
        majority_sign = 1

    d_joints = joint_coords[:,1] - (m*joint_coords[:,0] + b)

    plt.figure(figsize=(7,7))
    plt.plot(path_coords[:,0], path_coords[:,1], 'b-', label='Full curve')
    plt.plot(flat_pts[:,0], flat_pts[:,1], 'ro', label='Flat segment')
    plt.plot(x_line, y_line, 'k--', label='Best fit line (flat)')

    for i, (x, y) in enumerate(joint_coords):
        if np.sign(d_joints[i]) == majority_sign:
            plt.scatter(x, y, c='g', marker='x', s=80)
        else:
            plt.scatter(x, y, c='r', marker='x', s=80)
        plt.text(x, y, str(i), fontsize=9, ha='right', va='bottom')

    plt.axis('equal')
    plt.legend()
    plt.title(f"coords:{filename}")

    os.makedirs(save_dir, exist_ok=True)
    filename = f"{filename}.jpg"
    filepath = os.path.join(save_dir, filename)
    plt.savefig(filepath, format='jpg')
    plt.close()

def has_self_intersections(coords):
    line = LineString(coords)
    return not line.is_simple

def is_closed(pts):
    start_pt = pts[0]
    end_pt = pts[-1]
    path_dist = np.linalg.norm(end_pt - start_pt)
    is_closed = path_dist < 0.1
    return is_closed

def str_to_coords(s):
    s = str(s)
    nums = [float(x) for x in s.strip().split()]
    return [[nums[j], nums[j+1]] for j in range(0, len(nums), 2)]

# Worker function for parallel processing
def process_mechanism(args):
    mechType, initState, types, saveDir, saveDirWalking, saveDirWalkingPlot, numCoords, API_ENDPOINT, HEADERS = args
    
    coord = np.round(initState, 3).reshape((numCoords,2))
    dist = calc_dist(coord)  # Make sure calc_dist is defined or imported

    if max(dist) > 10:
        return None

    param = coord.tolist()
    name = str(param).replace("[", "").replace("]", "").replace(",", "") 

    exampleData = {
        'type': types[mechType], 
        'params': param,
        'speedScale': speedscale,  # Make sure speedscale is defined
        'steps': steps,  # Make sure steps is defined
        'relativeTolerance': 0.1 
    }

    # Process single mechanism
    try:
        response = requests.post(url=API_ENDPOINT, headers=HEADERS, data=json.dumps([exampleData])).json()
        time.sleep(0.02)
    except ValueError as v:
        for i in range(3):
            time.sleep(2)
            try:
                response = requests.post(url=API_ENDPOINT, headers=HEADERS, data=json.dumps([exampleData])).json()
                break
            except ValueError as v2:
                return None

    if not response:
        return None

    P = np.array(response[0]['poses'])
    result = None
    
    try:
        if len(P.shape) >= 1 and P.shape[0] >= minsteps:  # Make sure minsteps is defined
            imageMat, transParamSet, success = process_mech_051524(P, couplerCurveIndex[mechType])  # Make sure process_mech_051524 is defined
            
            if success:
                normalized_coords = normalize_data_122223(P[:,couplerCurveIndex[mechType],:], scaling=3.5)[0]  # Make sure normalize_data_122223 is defined
                
                pca_output = detect_flat_via_pca(normalized_coords)
                sharp, kappa = has_sharp_edges(normalized_coords, curvature_threshold=150.0)
                self_intersecting = has_self_intersections(resample_uniform(normalized_coords, 512))
                
                joint_coords = np.array(str_to_coords(name))
                ground_clearance = ground_clearance_check(
                    resample_uniform(P[:,couplerCurveIndex[mechType],:], 512),
                    joint_coords,
                    pca_output['flat_indices']
                )

                # Check walking criteria
                if (pca_output['is_flat_enough'] and is_closed(normalized_coords) and 
                    (not sharp) and ground_clearance and (not self_intersecting)):
                    
                    Tstr = np.array2string(np.round(transParamSet, 3), precision=3, suppress_small=True).replace("[", "").replace("]", "")
                    Tstr = re.sub('\s+', ' ', Tstr)
                    binary_data = np.uint8(imageMat[:,:,0]) * 255
                    img = Image.fromarray(binary_data)

                    # Save walking mechanism
                    img.save(saveDirWalking + "/" + name + ' ' + types[mechType] + ' ' + Tstr + '.jpg')
                    save_flat_fit_with_joints(
                        resample_uniform(P[:,couplerCurveIndex[mechType],:], 512), 
                        pca_output['flat_indices'], 
                        joint_coords, 
                        saveDirWalkingPlot, 
                        f"{name} {Tstr}"
                    )
                    
                    result = {
                        'type': types[mechType],
                        'params': param,
                        'flat_fraction': pca_output['flat_fraction'],
                        'max_run': pca_output['max_run'],
                        'orientation_deg': pca_output['orientation_deg'],
                        'sharp': sharp,
                        'ground_clearance': ground_clearance
                    }
                    
    except Exception as e:
        print(f"Error processing mechanism: {e}")
    
    return result

# Main parallel execution
normFailCtr = 0
mechType = 0
NUM_MECHS = 4000000

# Make sure these variables are defined in your environment:
# types, initStates, speedscale, steps, minsteps, couplerCurveIndex, calc_dist, process_mech_051524, normalize_data_122223

for index in range(0, 4):
    mechType = index
    print(f"Processing mechanism type: {types[mechType]}")
    
    # Set up directories (same as your original code)
    if mechType <= 3:
        saveDir = os.path.abspath("./outputs-4bar/" + types[mechType])
        saveDirNpy = os.path.abspath("./outputs-4bar/" + types[mechType] + "-npy")
        saveDirWalking = os.path.abspath("./outputs-4bar/walking_" + types[mechType])
        saveDirWalkingPlot = os.path.abspath("./outputs-4bar/walking_plot_" + types[mechType])
        numCoords = 5
    else:
        saveDir = os.path.abspath("./outputs-6bar/" + types[mechType])
        saveDirNpy = os.path.abspath("./outputs-6bar/" + types[mechType] + "-npy")
        saveDirWalking = os.path.abspath("./outputs-6bar/walking_" + types[mechType])
        saveDirWalkingPlot = os.path.abspath("./outputs-6bar/walking_plot_" + types[mechType])
        numCoords = 8

    if mechType in [8, 9, 10, 19, 22, 24]:
        numCoords = 7

    # Create directories
    for d in [saveDir, saveDirNpy, saveDirWalking, saveDirWalkingPlot]:
        os.makedirs(d, exist_ok=True)

    # Prepare data for parallel processing
    mechanism_data = initStates[mechType*NUM_MECHS: (mechType+1)*NUM_MECHS, :numCoords*2]
    
    # Create argument list for parallel processing
    args_list = []
    for initState in mechanism_data:
        args_list.append((
            mechType, initState, types, saveDir, saveDirWalking, 
            saveDirWalkingPlot, numCoords, API_ENDPOINT, HEADERS
        ))

    # Use multiprocessing with progress bar
    num_cores = mp.cpu_count()
    print(f"Using {num_cores} cores for parallel processing")
    
    with mp.Pool(processes=num_cores) as pool:
        results = []
        for result in tqdm(pool.imap_unordered(process_mechanism, args_list), 
                          total=len(args_list), 
                          desc=f"Processing {types[mechType]}"):
            if result is not None:
                results.append(result)
                print(f"\033[92mFound walking mechanism: {result}\033[0m")

    print(f"Completed {types[mechType]}. Found {len(results)} walking mechanisms.")

print("All mechanism types processed.")

Processing mechanism type: RRRR
Using 40 cores for parallel processing


Processing RRRR:   0%|          | 2989/4000000 [00:11<4:24:12, 252.14it/s]

Found walking mechanism: {'type': 'RRRR', 'params': [[4.781, -5.876], [6.677, -5.173], [8.369, 8.919], [0.849, 5.342], [0.527, 3.324]], 'flat_fraction': 0.37109375, 'max_run': 190, 'orientation_deg': 0.1676065093465736, 'sharp': np.False_, 'ground_clearance': np.True_}
Found walking mechanism: {'type': 'RRRR', 'params': [[2.238, 9.662], [3.284, 8.36], [2.533, -0.303], [-7.889, -3.411], [-8.062, 0.953]], 'flat_fraction': 0.30859375, 'max_run': 158, 'orientation_deg': -0.20319716254876413, 'sharp': np.False_, 'ground_clearance': np.True_}


Processing RRRR:   0%|          | 4132/4000000 [00:13<2:25:01, 459.24it/s]/tmp/ipykernel_1975539/4156772761.py:96: RankWarning: Polyfit may be poorly conditioned
  m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)
Processing RRRR:   0%|          | 4588/4000000 [00:14<1:55:41, 575.57it/s]

Found walking mechanism: {'type': 'RRRR', 'params': [[8.833, 0.867], [9.384, -0.59], [4.32, -9.622], [0.073, -6.76], [-1.529, -2.994]], 'flat_fraction': 0.404296875, 'max_run': 207, 'orientation_deg': 0.20733953375402517, 'sharp': np.False_, 'ground_clearance': np.True_}


...

Processing RRRR:  22%|██▏       | 895475/4000000 [26:57<1:37:24, 531.18it/s]/tmp/ipykernel_1975539/4156772761.py:96: RankWarning: Polyfit may be poorly conditioned
  m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)
Processing RRRR:  22%|██▏       | 896419/4000000 [26:59<1:37:15, 531.80it/s]

Found walking mechanism: {'type': 'RRRR', 'params': [[2.848, 5.978], [9.148, 9.677], [-9.681, -7.886], [5.897, -0.385], [9.236, -8.469]], 'flat_fraction': 0.28125, 'max_run': 144, 'orientation_deg': -179.9013894786401, 'sharp': np.False_, 'ground_clearance': np.True_}


Processing RRRR:  22%|██▏       | 897017/4000000 [27:00<1:35:24, 542.07it/s]

Found walking mechanism: {'type': 'RRRR', 'params': [[-6.756, -6.481], [-4.731, -6.83], [3.079, 9.064], [6.266, 3.008], [-2.651, 8.721]], 'flat_fraction': 0.421875, 'max_run': 216, 'orientation_deg': 0.16861905948254097, 'sharp': np.False_, 'ground_clearance': np.True_}


Processing RRRR:  23%|██▎       | 900219/4000000 [27:05<1:25:37, 603.42it/s]/tmp/ipykernel_1975539/4156772761.py:96: RankWarning: Polyfit may be poorly conditioned
  m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)
Processing RRRR:  23%|██▎       | 901277/4000000 [27:07<1:27:15, 591.86it/s]

In [8]:
print(normFailCtr) # basically not too many steps is the reason. 
print(P.shape)

NameError: name 'normFailCtr' is not defined